In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

## 1. Load Merged Dataset

In [2]:
# Load the merged dataset from previous step
df = pd.read_csv('/Users/priyankamalavade/Desktop/Audible_Insights_Project/data/merged_audible_dataset.csv')


print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print()

Dataset shape: (7576, 11)
Columns: ['Book_Name', 'Author', 'Rating', 'Number_of_Reviews', 'Price', 'Description', 'Listening_Time', 'Ranks_and_Genre', 'Has_Basic_Data', 'Has_Advanced_Data', 'Data_Source']



In [3]:
# Display basic info
print(" Dataset Overview:")
print(df.info())
print()

# Check missing values
print(" Missing Values Summary:")
missing_summary = df.isnull().sum()
print(missing_summary[missing_summary > 0])


 Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7576 entries, 0 to 7575
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Book_Name          7576 non-null   object 
 1   Author             7576 non-null   object 
 2   Rating             7576 non-null   float64
 3   Number_of_Reviews  6832 non-null   float64
 4   Price              7574 non-null   float64
 5   Description        5001 non-null   object 
 6   Listening_Time     5008 non-null   object 
 7   Ranks_and_Genre    5008 non-null   object 
 8   Has_Basic_Data     7576 non-null   bool   
 9   Has_Advanced_Data  7576 non-null   bool   
 10  Data_Source        7576 non-null   object 
dtypes: bool(2), float64(3), object(6)
memory usage: 547.6+ KB
None

 Missing Values Summary:
Number_of_Reviews     744
Price                   2
Description          2575
Listening_Time       2568
Ranks_and_Genre      2568
dtype: int64


## 2. Clean Basic Information


In [4]:
# Create a copy for cleaning
df_clean = df.copy()

# Clean book names - remove extra whitespace and standardize
df_clean['Book_Name'] = df_clean['Book_Name'].str.strip()
df_clean['Book_Name'] = df_clean['Book_Name'].str.replace(r'\s+', ' ', regex=True)

# Clean author names - remove extra whitespace and standardize
df_clean['Author'] = df_clean['Author'].str.strip()
df_clean['Author'] = df_clean['Author'].str.replace(r'\s+', ' ', regex=True)


In [5]:
# Remove any books without names or authors
initial_count = len(df_clean)
df_clean = df_clean.dropna(subset=['Book_Name', 'Author'])
df_clean = df_clean[df_clean['Book_Name'].str.len() > 0]
df_clean = df_clean[df_clean['Author'].str.len() > 0]



In [6]:
print(f" Removed {initial_count - len(df_clean)} books with missing names/authors")
print(f" Remaining books: {len(df_clean):,}")


 Removed 0 books with missing names/authors
 Remaining books: 7,576


## 3. Clean Numerical Data

In [7]:
# Clean Rating column
print(" Cleaning Ratings:")
print(f"Rating range before: {df_clean['Rating'].min():.2f} to {df_clean['Rating'].max():.2f}")
print(f"Missing ratings: {df_clean['Rating'].isnull().sum()}")


 Cleaning Ratings:
Rating range before: -1.00 to 5.00
Missing ratings: 0


In [8]:
# Ratings should be between 1.0 and 5.0
df_clean['Rating'] = pd.to_numeric(df_clean['Rating'], errors='coerce')
df_clean.loc[df_clean['Rating'] < 1.0, 'Rating'] = np.nan
df_clean.loc[df_clean['Rating'] > 5.0, 'Rating'] = np.nan

In [9]:
# Fill missing ratings with median rating
median_rating = df_clean['Rating'].median()
df_clean['Rating'].fillna(median_rating, inplace=True)
print(f" Filled missing ratings with median: {median_rating:.2f}")
print()

 Filled missing ratings with median: 4.50



In [10]:
# Clean Number of Reviews
print(" Cleaning Number of Reviews:")
print(f"Review count range before: {df_clean['Number_of_Reviews'].min():.0f} to {df_clean['Number_of_Reviews'].max():.0f}")
print(f"Missing review counts: {df_clean['Number_of_Reviews'].isnull().sum()}")


 Cleaning Number of Reviews:
Review count range before: 1 to 70122
Missing review counts: 744


In [11]:
# Convert to numeric and handle negative values
df_clean['Number_of_Reviews'] = pd.to_numeric(df_clean['Number_of_Reviews'], errors='coerce')
df_clean.loc[df_clean['Number_of_Reviews'] < 0, 'Number_of_Reviews'] = 0


In [12]:
# Fill missing review counts with 0 (assuming no reviews means 0 reviews)
df_clean['Number_of_Reviews'].fillna(0, inplace=True)
df_clean['Number_of_Reviews'] = df_clean['Number_of_Reviews'].astype(int)
print(" Filled missing review counts with 0")
print()

 Filled missing review counts with 0



In [13]:
# Clean Price column
print(" Cleaning Prices:")
print(f"Price range before: ${df_clean['Price'].min():.2f} to ${df_clean['Price'].max():.2f}")
print(f"Missing prices: {df_clean['Price'].isnull().sum()}")


 Cleaning Prices:
Price range before: $0.00 to $18290.00
Missing prices: 2


In [14]:
# Convert to numeric and handle outliers
df_clean['Price'] = pd.to_numeric(df_clean['Price'], errors='coerce')


In [15]:
# Remove extreme outliers (prices > $1000 might be data entry errors)
extreme_prices = df_clean['Price'] > 1000
print(f"Books with price > $1000: {extreme_prices.sum()}")


Books with price > $1000: 1540


In [16]:
# Cap extreme prices at 99th percentile
price_99th = df_clean['Price'].quantile(0.99)
df_clean.loc[df_clean['Price'] > price_99th, 'Price'] = price_99th


In [17]:
# Fill missing prices with median
median_price = df_clean['Price'].median()
df_clean['Price'].fillna(median_price, inplace=True)
print(f" Filled missing prices with median: ${median_price:.2f}")


 Filled missing prices with median: $680.00


## 4. Clean Listening Time

In [18]:
def parse_listening_time(time_str):
    """Convert listening time string to minutes"""
    if pd.isna(time_str) or time_str == '':
        return np.nan
    
    time_str = str(time_str).lower()
    total_minutes = 0
    
    # Extract hours
    hour_match = re.search(r'(\d+)\s*hour', time_str)
    if hour_match:
        total_minutes += int(hour_match.group(1)) * 60
    
    # Extract minutes
    minute_match = re.search(r'(\d+)\s*minute', time_str)
    if minute_match:
        total_minutes += int(minute_match.group(1))
    
    return total_minutes if total_minutes > 0 else np.nan

# Apply the parsing function
df_clean['Listening_Time_Minutes'] = df_clean['Listening_Time'].apply(parse_listening_time)


In [19]:
print(f"Successfully parsed listening times: {df_clean['Listening_Time_Minutes'].notna().sum()}")
print(f"Missing listening times: {df_clean['Listening_Time_Minutes'].isna().sum()}")


Successfully parsed listening times: 2354
Missing listening times: 5222


In [20]:
# Fill missing listening times with median
median_time = df_clean['Listening_Time_Minutes'].median()
df_clean['Listening_Time_Minutes'].fillna(median_time, inplace=True)


In [21]:
print(f" Filled missing listening times with median: {median_time:.0f} minutes")


 Filled missing listening times with median: 490 minutes


## 5. Clean Genre Information

In [22]:
def extract_genres(genre_str):
    """Extract clean genre list from ranks and genre string"""
    if pd.isna(genre_str) or genre_str == '':
        return []
    
    # Split by commas and clean each genre
    genres = []
    parts = str(genre_str).split(',')
    
    for part in parts:
        # Remove rankings like "#1 in" or "#2 in"
        cleaned = re.sub(r'#\d+\s+in\s+', '', part.strip())
        
        # Remove parenthetical information like "(See Top 100 in...)"
        cleaned = re.sub(r'\([^)]*\)', '', cleaned)
        
        # Clean whitespace
        cleaned = cleaned.strip()
        
        # Only keep non-empty genres
        if cleaned and len(cleaned) > 2:
            genres.append(cleaned)
    
    return genres

# Extract genres
df_clean['Genres'] = df_clean['Ranks_and_Genre'].apply(extract_genres)


In [23]:
# Count how many books have genres
books_with_genres = df_clean['Genres'].apply(len) > 0
print(f"Books with genre information: {books_with_genres.sum()}")


Books with genre information: 2359


In [24]:
# Get all unique genres
all_genres = []
for genre_list in df_clean['Genres']:
    all_genres.extend(genre_list)

unique_genres = list(set(all_genres))
print(f"Total unique genres found: {len(unique_genres)}")
print()

Total unique genres found: 2648



In [28]:
# Display most common genres
from collections import Counter
genre_counts = Counter(all_genres)
print(" Top 20 Most Common Genres:")
for genre, count in genre_counts.most_common(20):
    print(f"  {genre}: {count}")


 Top 20 Most Common Genres:
  Audible Audiobooks & Originals: 602
  Personal Success: 222
  Classic Literature: 94
  Leadership: 83
  Literary Fiction: 75
  Thriller & Mystery: 71
  Historical Fiction: 65
  Meditation: 56
  Contemporary Romance: 53
  Personal Finance: 50
  Business Management: 50
  Psychology: 49
  Business Careers: 47
  Body & Spirit: 47
  5 star: 45
  Entrepreneurship: 43
  Spirituality: 43
  Business Motivation & Self-Improvement: 39
  Analysis & Strategy: 37
  Forecasting & Strategic Planning: 36


## 6. Clean Description Text

In [29]:
def clean_description(desc):
    """Clean and standardize book descriptions"""
    if pd.isna(desc) or desc == '':
        return ''
    
    desc = str(desc)
    
    # Remove extra whitespace
    desc = re.sub(r'\s+', ' ', desc)
    
    # Remove leading/trailing whitespace
    desc = desc.strip()
    
    # Remove any HTML tags if present
    desc = re.sub(r'<[^>]+>', '', desc)
    
    return desc

# Clean descriptions
df_clean['Description_Clean'] = df_clean['Description'].apply(clean_description)


In [30]:
# Calculate description lengths
df_clean['Description_Length'] = df_clean['Description_Clean'].str.len()


In [31]:
descriptions_available = df_clean['Description_Length'] > 0
print(f"Books with descriptions: {descriptions_available.sum()}")
print(f"Average description length: {df_clean['Description_Length'].mean():.0f} characters")
print(f"Median description length: {df_clean['Description_Length'].median():.0f} characters")


Books with descriptions: 4999
Average description length: 153 characters
Median description length: 122 characters


## 7. Create Derived Features

In [32]:
# Popularity score based on number of reviews
df_clean['Popularity_Score'] = np.log1p(df_clean['Number_of_Reviews'])


In [34]:
df_clean['Popularity_Score'].head(5)

0    5.141664
1    5.141664
2    7.728856
3    7.728856
4    5.192957
Name: Popularity_Score, dtype: float64

In [35]:
# Rating category
def categorize_rating(rating):
    if rating >= 4.5:
        return 'Excellent'
    elif rating >= 4.0:
        return 'Very Good'
    elif rating >= 3.5:
        return 'Good'
    elif rating >= 3.0:
        return 'Average'
    else:
        return 'Below Average'

df_clean['Rating_Category'] = df_clean['Rating'].apply(categorize_rating)


In [36]:
# Price category
price_quantiles = df_clean['Price'].quantile([0.25, 0.5, 0.75])
def categorize_price(price):
    if price <= price_quantiles[0.25]:
        return 'Low'
    elif price <= price_quantiles[0.5]:
        return 'Medium-Low'
    elif price <= price_quantiles[0.75]:
        return 'Medium-High'
    else:
        return 'High'

df_clean['Price_Category'] = df_clean['Price'].apply(categorize_price)


In [37]:
# Length category for listening time
def categorize_length(minutes):
    if minutes < 180:  # Less than 3 hours
        return 'Short'
    elif minutes < 480:  # Less than 8 hours
        return 'Medium'
    elif minutes < 900:  # Less than 15 hours
        return 'Long'
    else:
        return 'Very Long'

df_clean['Length_Category'] = df_clean['Listening_Time_Minutes'].apply(categorize_length)


In [38]:
print(" Created derived features:")
print(f"  - Popularity Score (log of reviews + 1)")
print(f"  - Rating Category: {df_clean['Rating_Category'].value_counts().to_dict()}")
print(f"  - Price Category: {df_clean['Price_Category'].value_counts().to_dict()}")
print(f"  - Length Category: {df_clean['Length_Category'].value_counts().to_dict()}")


 Created derived features:
  - Popularity Score (log of reviews + 1)
  - Rating Category: {'Excellent': 4878, 'Very Good': 2292, 'Good': 281, 'Average': 81, 'Below Average': 44}
  - Price Category: {'Low': 2072, 'Medium-High': 2020, 'High': 1762, 'Medium-Low': 1722}
  - Length Category: {'Long': 6025, 'Medium': 837, 'Very Long': 396, 'Short': 318}


## 8. Final Data Quality Check

In [39]:
# Check for any remaining issues
print(" Missing values after cleaning:")
missing_after = df_clean.isnull().sum()
print(missing_after[missing_after > 0])
print()

 Missing values after cleaning:
Description        2575
Listening_Time     2568
Ranks_and_Genre    2568
dtype: int64



In [40]:
# Check for duplicates
duplicates = df_clean.duplicated(subset=['Book_Name', 'Author']).sum()
print(f" Duplicate book-author combinations: {duplicates}")


 Duplicate book-author combinations: 1511


In [41]:
if duplicates > 0:
    print("Removing duplicates...")
    df_clean = df_clean.drop_duplicates(subset=['Book_Name', 'Author'], keep='first')
    print(f" Removed {duplicates} duplicates")


Removing duplicates...
 Removed 1511 duplicates


In [42]:
print()
print(" Final dataset statistics:")
print(f"  - Total books: {len(df_clean):,}")
print(f"  - Books with descriptions: {(df_clean['Description_Length'] > 0).sum():,}")
print(f"  - Books with genres: {(df_clean['Genres'].apply(len) > 0).sum():,}")
print(f"  - Average rating: {df_clean['Rating'].mean():.2f}")
print(f"  - Average reviews: {df_clean['Number_of_Reviews'].mean():.0f}")
print(f"  - Average price: ${df_clean['Price'].mean():.2f}")



 Final dataset statistics:
  - Total books: 6,065
  - Books with descriptions: 4,005
  - Books with genres: 2,047
  - Average rating: 4.46
  - Average reviews: 841
  - Average price: $898.11


## 9. Data Type Optimization

In [43]:
# Optimize data types to save memory
df_clean = df_clean.astype({
    'Number_of_Reviews': 'int32',
    'Listening_Time_Minutes': 'float32',
    'Description_Length': 'int32',
    'Popularity_Score': 'float32',
    'Rating': 'float32',
    'Price': 'float32'
})

In [44]:
# Convert categorical columns to category type
categorical_columns = ['Rating_Category', 'Price_Category', 'Length_Category', 'Data_Source']
for col in categorical_columns:
    df_clean[col] = df_clean[col].astype('category')


In [45]:
print(" Optimized data types for better memory usage")
print()
print(" Final dataset info:")
print(df_clean.info(memory_usage='deep'))


 Optimized data types for better memory usage

 Final dataset info:
<class 'pandas.core.frame.DataFrame'>
Index: 6065 entries, 0 to 7572
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   Book_Name               6065 non-null   object  
 1   Author                  6065 non-null   object  
 2   Rating                  6065 non-null   float32 
 3   Number_of_Reviews       6065 non-null   int32   
 4   Price                   6065 non-null   float32 
 5   Description             4007 non-null   object  
 6   Listening_Time          4013 non-null   object  
 7   Ranks_and_Genre         4013 non-null   object  
 8   Has_Basic_Data          6065 non-null   bool    
 9   Has_Advanced_Data       6065 non-null   bool    
 10  Data_Source             6065 non-null   category
 11  Listening_Time_Minutes  6065 non-null   float32 
 12  Genres                  6065 non-null   object  
 13  Description_Cle

## 10. Save Cleaned Dataset

In [46]:
# Prepare final columns for saving
final_columns = [
    'Book_Name', 'Author', 'Rating', 'Number_of_Reviews', 'Price',
    'Description_Clean', 'Listening_Time_Minutes', 'Genres',
    'Rating_Category', 'Price_Category', 'Length_Category',
    'Popularity_Score', 'Description_Length', 'Data_Source'
]

In [47]:
df_final = df_clean[final_columns].copy()

# Save cleaned dataset
output_filename = '/Users/priyankamalavade/Desktop/Audible_Insights_Project/data/cleaned_audible_dataset.csv'
df_final.to_csv(output_filename, index=False)


In [48]:
print(f" Cleaned dataset saved as: {output_filename}")
print(f" Final dataset shape: {df_final.shape}")
print()


 Cleaned dataset saved as: /Users/priyankamalavade/Desktop/Audible_Insights_Project/data/cleaned_audible_dataset.csv
 Final dataset shape: (6065, 14)



In [52]:
# Display sample of cleaned data
print(" Sample of cleaned dataset:")
sample_cols = ['Book_Name', 'Author', 'Rating', 'Number_of_Reviews', 'Price', 'Rating_Category']
df_final[sample_cols].head(10)


 Sample of cleaned dataset:


,Book_Name,Author,Rating,Number_of_Reviews,Price,Rating_Category
0,"""Don't You Know Who I Am?"": How to Stay Sane i...",Ramani S. Durvasula PhD,4.8,170,836.0,Excellent
2,#Girlboss,Sophia Amoruso,4.5,2272,615.0,Excellent
4,#TheRealCinderella: #BestFriendsForever Series...,Yesenia Vargas,4.3,179,586.0,Very Good
5,10 Bedtime Stories For Little Kids,div.,4.5,0,376.0,Excellent
7,10 Essential Pieces of Literature,Khalil Gibran,4.5,0,32.0,Excellent
9,10 Essential Success Mantras from the Bhagavad...,Vimla Patil,4.2,45,233.0,Very Good
10,10 Judgements That Changed India,Zia Mody,4.4,221,502.0,Very Good
11,10 Masterpieces You Have to Read Before You Die 1,Jane Austen,4.4,344,401.0,Very Good
12,10 Minutes 38 Seconds in This Strange World,Elif Shafak,4.5,520,752.0,Excellent
13,10 Skills for Effective Business Communication...,Jessica Higgins JD MBA BB,4.6,45,501.0,Excellent
